In [2]:
import pandas as pd
import numpy as np
from collections import Counter
import time
import datetime

In [3]:
seats = pd.read_csv("Data/seats.csv").drop(columns="Unnamed: 0")
seats.head(2)

,ID,Type,x,y,z,Area,Window,Computer,xy array,dist_toilet,dist_elevator,dist_stairs,taken,code
0,1,seat,0.5,0.5,0,A,1,1,[0.5 0.5],3.535534,3.535534,4.743416,0,9191
1,2,seat,1.5,0.5,0,A,1,1,[1.5 0.5],2.549510,2.549510,3.807887,0,5741


In [4]:
students = pd.read_pickle("Data/students.p")
students.head(2)

,student_id,password,total_time_spent,past_seats,past_books,current_seat,time_spent,email
0,123456,1234560,0,[],[],0,0,123456@unilibrary.com
1,123457,1234570,0,[],[],0,0,123457@unilibrary.com


### reserve seat

In [5]:
possible_seats = []

In [6]:
def reserve_seat():
    """
    Function that asks for login data and continues to functions that filter for seats.
    """
    global studentid

    studentid = int(input("Please enter your student ID."))
    password = int(input("Please enter your passwords."))
 
    if password == int(students.loc[students["student_id"] == studentid, "password"]):
        print("There are", len(seats[seats["taken"]==0]), "seats available.")
        filtering()
        
    else: 
        print("Student ID or password incorrect. Please try again.")
        reserve_seat()


In [7]:
def filtering():
    """
    Function to decide wether or not to filter the availabe seats.
    """
    filt = input("Do you want to filter the available seats?").lower()
    
    if filt == "yes":
        choose_filter()

    elif filt == "no":
        if len(possible_seats) == 0:
            seat = seats.loc[seats['taken'] == 0, 'ID']
            for i in seat:
                possible_seats.append(i)
        best_seats()
        
    else:
        print("Invalid entry. Please enter yes or no")
        filtering()


In [8]:
def best_seats():
    """
    Function that presents the best seats and lets you choose one that will be reserved.
    """
    global chosen_seat

    seat_freq = dict(Counter(possible_seats))
    seat_dict = seat_freq
    max_value = max(seat_freq.values())
    choice = [k for k,v in seat_dict.items() if v == max_value]

    print("Following seats are optimal for you:", choice)

    chosen_seat = int(input("Please enter the ID of the place you want to reserve."))

    if chosen_seat in choice:

        seats.loc[seats.ID == chosen_seat, 'taken'] = 1
        students.loc[students.student_id == studentid, 'current_seat'] = chosen_seat
        print("Your seat with the ID" , chosen_seat , "has been reserved.")
        reserve_time()
        
    else:
        print("This seat is not available. Make sure that you enter the correct ID.")
        best_seats()

In [16]:
def reserve_time():
    """
    Function to reserve seat for a certain amount of time
    """
    print("For how long do you want to reserve the seat? Please enter hours, minutes and seconds.")
    h = int(input("Enter hours."))
    m = int(input("Enter minutes."))
    s = int(input("Enter seconds."))
    total_seconds = h * 3600 + m * 60 + s

    while total_seconds > -1:
        timer = datetime.timedelta(seconds = total_seconds)
        print(timer, end="\r")
        time.sleep(1)
        total_seconds -= 1
        
    seats.loc[seats.ID == chosen_seat, 'taken'] = 0
    students.loc[students.student_id == studentid, 'current_seat'] = 0
    #students.loc[students.student_id == studentid, "time_spent"] = total_seconds# - time passed

In [90]:
def choose_filter():
    """
    Function to choose filter.
    """
    filt = input("Choose a filter: section, floor, window, computer, toilet, elevator and stairs.").lower()
    if filt == "section":
        section()
    elif filt == "floor":
        floor()
    elif filt == "window":
        window()
    elif filt == "computer":
        computer()
    elif filt == "toilet":
        toilet()
    elif filt == "elevator":
        elevator()
    elif filt == "stairs":
        stairs()
    else:
        print("Invalid entry. This category does not exist.")
        choose_filter()
    filtering()

In [127]:
def section():
    """
    Function to filter seats regarding their section in the library.
    """
    s = input("Which section do you prefer?").upper()
    lis_s = ["A", "B", "C", "D", "E", "F", "G", "H"]

    if s in lis_s:
        df = seats[seats["Area"]==s]
        seat = df.loc[df['taken'] == 0, 'ID']

        for i in seat:
            possible_seats.append(i)

        another = input("Do you want to choose another section?").lower()
        
        if another == "yes":
            section()
            
    else:
        print("Invalid entry. We don not have seats in this area.")
        section()
    

In [92]:
def floor():
    """
    Function that filters seats regarding preferred floor.
    """
    f = input("Which floor do you prefer: ground floor or first floor?").lower()

    if f == "ground floor":
        df = seats[seats["z"]==0]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)

    elif f == "first floor":
        df = seats[seats["z"]==1]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    else:
        print("Invalid entry.")
        

In [94]:
def window():
    """
    Function to filter wether or not a place should be located close to the window.
    """
    w = input("Do you want to sit close to a window?").lower().strip()
    
    if w == "yes":
        df = seats[seats["Window"]==1]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    elif w == "no":
        df = seats[seats["Window"]==0]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    else:
        print("Invalid entry. Please type yes or no.")
        window()

In [95]:
def computer():
    """
    Function to filter wether or not a computer is wanted.
    """
    c = input("Do you want a place with a computer?").lower().strip()
    
    if c == "yes":
        df = seats[seats["Computer"]==1]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    elif c == "no":
        df = seats[seats["Computer"]==0]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    else:
        print("Invalid entry. Please type yes or no.")
        computer()


In [96]:
def toilet():
    """
    Function to filter wether or not the seat should be close tot the toilets.
    """
    t = input("Do you want to sit close to or far from the toilet? Type close or far").lower().strip()
    
    if t == "close":
        df = seats[seats["dist_toilet"]<=5]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    elif t == "far":
        df = seats[seats["dist_toilet"]>=6]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    else:
        print("Invalid entry.")
        toilet()

In [97]:
def elevator():
    """
    Function to filter wether or not the seat should be close tot the elevator.
    """
    e = input("Do you want to sit close to or far from the elevator? Type close or far").lower().strip()
    
    if e == "close":
        df = seats[seats["dist_elevator"]<=5]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    elif e == "far":
        df = seats[seats["dist_elevator"]>=6]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    else:
        print("Invalid entry.")
        elevator()

In [98]:
def stairs():
    """
    Function to filter wether or not the seat should be close tot the stairs.
    """
    s = input("Do you want to sit close to or far from the stairs? Type close or far").lower().strip()
    
    if s == "close":
        df = seats[seats["dist_stairs"]<=5]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    elif s == "far":
        df = seats[seats["dist_stairs"]>=6]
        seat = df.loc[df['taken'] == 0, 'ID']
        for i in seat:
            possible_seats.append(i)
    
    else:
        print("Invalid entry.")
        stairs()

### take a break

In [102]:
seats.head(2)

,ID,Type,x,y,z,Area,Window,Computer,xy array,dist_toilet,dist_elevator,dist_stairs,taken,code
0,1,seat,0.5,0.5,0,A,1,1,[0.5 0.5],3.535534,3.535534,4.743416,0,9191
1,2,seat,1.5,0.5,0,A,1,1,[1.5 0.5],2.549510,2.549510,3.807887,0,5741


In [114]:
def take_a_break():
    """
    Function that allows you to reserve a seat for taking a break.
    """
    global stud_id

    stud_id = int(input("Please enter your student ID."))
    password = int(input("Please enter your passwords."))
 
    if password == int(students.loc[students["student_id"] == stud_id, "password"]):
        
        if int(students.loc[students["student_id"] == stud_id, "current_seat"]) != 0:
            block_seat()

        else:
            print("You do not have a place in the library at the moment.")
    else: 
        print("Student ID or password incorrect. Please try again.")
        take_a_break()

In [121]:
def block_seat():
    """
    Block seat while taking a break.
    """
    code = int(input("Enter the code of the table you are currently sitting at."))

    stud_seat = int(students.loc[students["student_id"] == stud_id, "current_seat"])
    
    if code == int(seats.loc[seats["ID"] == stud_seat, "code"]):
        reserve_time()
    else:
        print("Invalid entry.")
        block_seat()

In [118]:
reserve_seat()

There are 104 seats available.
Invalid entry. We don not have seats in this area.
Invalid entry. We don not have seats in this area.
Invalid entry. We don not have seats in this area.
Invalid entry. Please type yes or no.
Following seats are optimal for you: [1, 2, 3, 4, 5, 9, 53, 54, 55, 56, 57, 61]
Your seat with the ID 5 has been reserved.
For how long do you want to reserve the seat? Please enter hours, minutes and seconds.


In [ ]:
take_a_break()